In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [3]:
df = pd.read_csv('2020.12.13 boiling_plan.csv')
df['sku'] = df['sku'].apply(cast_sku)
df['boiling'] = df['sku'].apply(lambda sku: sku.boilings[0])
df.sort_values(by='id')
df['packing_speed'] = df['sku'].apply(lambda sku: sku.packing_speed)
df['boiling_form_factor'] = df['sku'].apply(lambda sku:sku.boiling_form_factors[0])

df.at[:2, 'packing_team_id'] = 1
df.at[2:, 'packing_team_id'] = 2
boiling_plan_df = df.copy()
boiling_plan_df.head()

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id
0,1.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0


In [4]:
def gen_boiling_plan(boiling_plan_df, id):
    boiling_plan = boiling_plan_df[boiling_plan_df['id'] == id]

    # mark groups
    cur_group = 0
    cur_form_factor = None
    group_values = []
    for i, row in boiling_plan.iterrows():
        if row['boiling_form_factor'] != cur_form_factor:
            cur_group += 1
            cur_form_factor = row['boiling_form_factor']
        group_values.append(cur_group)
    boiling_plan['group'] = group_values
    return boiling_plan

# Create schedule for sample boiling

In [5]:
def make_boiling(boiling_plan):
    boiling_model = boiling_plan.iloc[0]['boiling']
    
    # todo: del
    if boiling_model.boiling_type == 'salt':
        boiling_model.meltings.speed = 850 / 50 * 60

    maker, root, make = new_block_maker(default_push_func=dummy_push)

    # [termizator.time]
    # todo: del
    termizator = db.session.query(Termizator).first()
    termizator.pouring_time = 30
    
    with make('boiling', boiling_model=boiling_model):
        with make('pouring', push_func=add_push):
            with make('first'):
                make('termizator', time_size=termizator.pouring_time)
                make('fermenting', time_size=boiling_model.pourings.pouring_time - termizator.pouring_time)
                make('soldification', time_size=boiling_model.pourings.soldification_time)
                make('cutting', time_size=boiling_model.pourings.cutting_time)
            with make('second'):
                make('pouring_off', time_size=boiling_model.pourings.pouring_off_time)
                make('extra', time_size=boiling_model.pourings.extra_time)

        make('drenator', 
             t=root['boiling']['pouring']['first'].end(),
             time_size=boiling_model.lines.cheddarization_time * 12 * 5, 
             push_func=add_push)

        with make('melting'):
            serving = make('serving', push_func=add_push, time_size=boiling_model.meltings.serving_time).block

            meltings = make('meltings', push_func=add_push, t=serving.length()).block
            coolings = make('coolings', push_func=add_push, t=serving.length()).block

            for group, grp in boiling_plan.groupby('group'):
                if group > 1 and boiling_model.boiling_type == 'water':
                    # non-first group - reconfigure time
                    dummy_push(meltings, Block('melting_configuration', size=1))

                melting_process = Block('melting_process',
                                        time_size=custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil'),
                                       actual_time_size=grp['kg'].sum() / boiling_model.meltings.speed * 60)

                dummy_push(meltings, melting_process)

                # add cooling
                cooling_process = Block('cooling_process', t=melting_process.props.relative_props['t'])
                cooling_maker, _, cooling_make = new_block_maker(cooling_process, default_push_func=dummy_push)

                with cooling_make('start'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=boiling_model.meltings.first_cooling_time)
                        cooling_make('cooling', time_size=boiling_model.meltings.second_cooling_time)
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=boiling_model.meltings.salting_time)

                with cooling_make('finish'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=melting_process.props['time_size'])
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=melting_process.props['time_size'])

                add_push(coolings, cooling_process)
                
    return root['boiling']

In [6]:
boilings = []
ids = list(boiling_plan_df['id'].unique())[:2]
for id in ids:
    boiling_plan = gen_boiling_plan(boiling_plan_df, id)
    boiling = make_boiling(boiling_plan)
    print('Boiling', id, boiling.props['boiling_model'].boiling_type)
    print(draw_print(boiling))
    boilings.append(boiling)


Boiling 1.0 water
================================================================================================== boiling 0:[0,98)
=========================== pouring 0:[0,27)
====================== first 0:[0,22)
====== termizator 0:[0,6)
      == fermenting 0:[6,8)
        ======= soldification 0:[8,15)
               ======= cutting 0:[15,22)
                      ===== second 0:[22,27)
                      === pouring_off 0:[22,25)
                         == extra 0:[25,27)
                      ================================================ drenator 0:[22,70)
                                                                      ============================ melting 0:[70,98)
                                                                      ====== serving 0:[70,76)
                                                                            ============== meltings 0:[76,90)
                                                                            ============== melting_p

<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values


# Combine boilings

In [16]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return

    boilings = [node for node in parent.children if node.props['class'] == 'boiling']

    b2 = boiling

    for b1 in reversed(boilings[-6:]): # compare with four previous boilings
        validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])  # [termizator.basic]

        # cannot make two boilings on same line at the same time
        if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
            validate_disjoint(b1['pouring'], b2['pouring'])

        if b1.props['boiling_type'] == b2.props['boiling_type']:
            # [melting.disjoint]
            validate_disjoint(b1['melting']['meltings'], b2['melting']['meltings'])

In [17]:
boiling_plan.head()

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id,group
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1


In [54]:
lines_df = pd.DataFrame(index=['water', 'salt'], columns=['iter_props', 'start_time', 'boiling_ids_left', 'latest_boiling'])
lines_df.at['water', 'iter_props'] = [{'pouring_line': str(v)} for v in [0, 1]]
lines_df.at['salt', 'iter_props'] = [{'pouring_line': str(v)} for v in [2, 3]]

# [cheesemakers.start_time]
lines_df.at['water', 'start_time'] = '09:50'
lines_df.at['salt', 'start_time'] = '07:05'

for boiling_type in ['water', 'salt']:
    ids = boiling_plan_df[boiling_plan_df['boiling'].apply(lambda b: b.boiling_type) == boiling_type].id
    ids = list(ids)
    ids = remove_duplicates(ids)
    lines_df.at[boiling_type, 'boiling_ids_left'] = ids
    
lines_df['latest_boiling'] = None
lines_df

,iter_props,start_time,boiling_ids_left,latest_boiling
water,"[{'pouring_line': '0'}, {'pouring_line': '1'}]",09:50,"[1.0, 2.0, 3.0, 4.0, 5.0]",None
salt,"[{'pouring_line': '2'}, {'pouring_line': '3'}]",07:05,"[6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 15.0, 16.0, 17.0]",None


In [55]:

schedule = Block('schedule')

def add_block(boiling_type):
    boiling_id = lines_df.at[boiling_type, 'boiling_ids_left'].pop(0)
    boiling_plan = gen_boiling_plan(boiling_plan_df, boiling_id)
    boiling = make_boiling(boiling_plan)

    if not lines_df.at[boiling_type, 'latest_boiling']:
        beg = cast_t(lines_df.at[boiling_type, 'start_time']) - boiling['melting'].beg()
    else:
        beg = lines_df.at[boiling_type, 'latest_boiling'].x1

    dummy_push(schedule, boiling, iter_props=lines_df.at[boiling_type, 'iter_props'], validator=boiling_validator, beg=beg, max_tries=100)

    lines_df.at[boiling_type, 'latest_boiling'] = boiling
    return boiling

In [56]:
add_block('water');
# add_block('salt')
# add_block('salt')
# add_block('water')

<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values


# Draw boilings

In [75]:
def add_push_with_props(parent, block, **kwargs):
    new_props = block.props.get_all_props()
    new_props.update(kwargs)
    return add_push(parent, block, new_props=new_props)

maker, front, make = new_block_maker(default_push_func=dummy_push)

make('stub', h=1, push_func=dummy_push_y)

with make('header', h=1, index_width=3, push_func=dummy_push_y, visible=False):
    make(h=1, size=1, text='График наливов', push_func=add_push)
    for i in range(288):
        cur_time = cast_time(i + cast_t('01:00'))
        if cur_time[-2:] == '00':
            make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
        else:
            make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)

boiling_type = 'water'
n_cooling_lines = 4
with make('line', h=2 * (n_cooling_lines + 2) + 1, push_func=dummy_push_y):
    for i in range(2):
        with make(f'pouring_line_{i}', h=n_cooling_lines + 1, push_func=dummy_push_y):
            with make(h=2, push_func=dummy_push_y):
                for boiling in [boiling for boiling in listify(schedule['boiling']) if boiling.props['pouring_line'] == str(i)]:
                    make(boiling['pouring'], push_func=add_push_with_props, push_kwargs={'y': 0})
                    make(boiling['drenator'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
                    make(boiling['melting']['serving'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
                    make(boiling['melting']['meltings'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
            
            cooling_lines = [make(f'cooling_{i}', h=1, push_func=dummy_push_y).block for i in range(n_cooling_lines)]

            cur_i = 0
            for boiling in [boiling for boiling in listify(schedule['boiling']) if boiling.props['boiling_model'].boiling_type == boiling_type and boiling.props['pouring_line'] == str(i)]:
                for cooling_process in listify(boiling['melting']['coolings']['cooling_process']):
                    for cooling in cooling_process.iter():
                        if cooling.props['class'] in ['cooling', 'salting']:
                            add_push_with_props(cooling_lines[cur_i % n_cooling_lines], cooling)
                    cur_i += 1
        make('stub', h=1, push_func=dummy_push_y)

In [76]:
front['line']

line (2, 15] x (0, 146]
  pouring_line_0 (2, 7] x (0, 146]
    block (2, 4] x (0, 138]
      pouring (2, 3] x (48, 75]
        first (2, 3] x (48, 70]
          termizator (2, 3] x (48, 54]
          fermenting (2, 3] x (54, 56]
          soldification (2, 3] x (56, 63]
          cutting (2, 3] x (63, 70]
        second (2, 3] x (70, 75]
          pouring_off (2, 3] x (70, 73]
          extra (2, 3] x (73, 75]
      drenator (3, 4] x (70, 118]
      serving (3, 4] x (118, 124]
      meltings (3, 4] x (124, 138]
        melting_process (3, 4] x (124, 138]
    cooling_0 (4, 5] x (0, 146]
      cooling (36, 37] x (124, 128]
      cooling (36, 37] x (128, 132]
      cooling (36, 37] x (132, 146]
    cooling_1 (5, 6] x (0, 0]
    cooling_2 (6, 7] x (0, 0]
    cooling_3 (7, 8] x (0, 0]
  stub (7, 8] x (0, 0]
  pouring_line_1 (8, 13] x (0, 0]
    block (8, 10] x (0, 0]
    cooling_0 (10, 11] x (0, 0]
    cooling_1 (11, 12] x (0, 0]
    cooling_2 (12, 13] x (0, 0]
    cooling_3 (13, 14] x (0, 

In [77]:
style = load_style()
wb = draw_schedule(front, style, init_sheet_func=init_sheet)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 

# План
2. Сделать объединенные варки с фасовкой
3. Нарисовать полное расписание
4. Добавить перенастройки и мойки